# Analysis of the effect of starting locations on angular error in HumanA
author: Nora Maleki (noora.maleki@gmail.com)
April 2022

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pingouin as pg
from statsmodels.stats.anova import AnovaRM

In [2]:
colorblind_palette = ['#2d004b', '#5ab4ac', '#b2182b', '#8c510a','#4575b4','#8073ac']
sns.set_theme(font_scale=1, palette=colorblind_palette, style='darkgrid')

## Load data

In [3]:
FILE_PATH = r'../../data/01_main'
PLOT_PATH = f'{FILE_PATH}/plots/analysis/'

os.makedirs(os.path.dirname(PLOT_PATH), exist_ok=True)

In [4]:
df_cleaned = pd.read_csv(f'{FILE_PATH}/07_preprocessed.csv')
df_cleaned.head()

,subject_id,body_pos_x,body_pos_z,pointer_pos_x,pointer_pos_z,pointer_dir_x,pointer_dir_z,building_center_z,building_center_x,avatar_presence,...,moved_target_position_z,target_to_x_180,dir_to_x_180,target_to_x_360,dir_to_x_360,final_180_angles,final_360_angles,rounded_180_angles,rounded_360_angles,absolute_180_angles
0,365,-114.329147,-139.997375,-114.236984,-139.696228,-0.055459,0.989548,53.288437,-72.578285,True,...,193.285812,77.811048,93.207763,77.811048,93.207763,15.396716,15.396716,15.0,15.0,15.396716
1,365,-65.669235,-159.945099,-65.474716,-159.710205,0.072550,0.992235,53.288437,-72.578285,True,...,213.233536,91.855810,85.818077,91.855810,85.818077,-6.037733,353.962267,-6.0,354.0,6.037733
2,365,38.849380,53.547638,38.471634,53.516281,-0.993265,-0.112792,53.288437,-72.578285,True,...,-0.259201,-179.866720,-173.521430,180.133280,186.478570,6.345290,6.345290,6.0,6.0,6.345290
3,365,38.880081,53.564148,38.700195,53.581047,-0.995648,-0.065598,53.288437,-72.578285,True,...,-0.275711,-179.858270,-176.230517,180.141730,183.769483,3.627752,3.627752,4.0,4.0,3.627752
4,365,110.754181,196.522675,110.428146,196.550751,-0.941145,-0.321944,53.288437,-72.578285,True,...,-143.234238,-142.000152,-161.115388,217.999848,198.884612,-19.115237,340.884763,-19.0,341.0,19.115237


In [69]:
anova_df = df_cleaned[['subject_id', 'starting_loc_id', 'avatar_id','absolute_180_angles']]
# anova_df['subject_id'] = anova_df.subject_id.astype(str)
# anova_df['starting_loc_id'] = anova_df.starting_loc_id.astype(str)
# anova_df['avatar_id'] = anova_df.avatar_id.astype(str)
anova_df.head()

,subject_id,starting_loc_id,avatar_id,absolute_180_angles
0,365,44,46,15.396716
1,365,36,46,6.037733
2,365,30,46,6.345290
3,365,30,46,3.627752
4,365,19,46,19.115237


In [70]:
pg.sphericity(data=anova_df, dv='absolute_180_angles', subject='subject_id', within='starting_loc_id')

SpherResults(spher=True, W=1.4454357702487354e-28, chi2=373.14842818826077, dof=377, pval=7.173929371629949)

In [71]:
anova = pg.rm_anova(dv='absolute_180_angles', within='starting_loc_id', subject='subject_id', data=anova_df)
anova

,Source,ddof1,ddof2,F,p-unc,np2,eps
0,starting_loc_id,27,405,5.737237,2.454395e-16,0.276664,0.340751


In [72]:
anova_df_agg = anova_df.groupby(['subject_id', 'starting_loc_id']).absolute_180_angles.mean().reset_index()
anova_df_agg.head()

,subject_id,starting_loc_id,absolute_180_angles
0,365,1,28.290829
1,365,2,21.913651
2,365,4,48.221470
3,365,5,67.533743
4,365,7,42.938610


In [73]:
print(AnovaRM(data=anova_df_agg, depvar='absolute_180_angles', subject='subject_id', within=['starting_loc_id']).fit())

                     Anova
                F Value  Num DF  Den DF  Pr > F
-----------------------------------------------
starting_loc_id  5.7372 27.0000 405.0000 0.0000



Building

In [74]:
pg.sphericity(data=anova_df, dv='absolute_180_angles', subject='subject_id', within='avatar_id')

SpherResults(spher=True, W=4.958725591338539e-105, chi2=-1082.2218644633415, dof=1539, pval=1.0)

In [75]:
pg.rm_anova(dv='absolute_180_angles', within='avatar_id', subject='subject_id', data=anova_df)

,Source,ddof1,ddof2,F,p-unc,np2,eps
0,avatar_id,55,770,2.984731,2.737571e-11,0.17573,0.198601


In [76]:
anova_df_agg_avatar = anova_df.groupby(['subject_id', 'avatar_id']).absolute_180_angles.mean().reset_index()
anova_df_agg_avatar.head()

,subject_id,avatar_id,absolute_180_angles
0,365,1,50.278107
1,365,2,61.830706
2,365,3,44.135103
3,365,4,55.623495
4,365,5,55.204059


In [82]:
print(AnovaRM(data=anova_df_agg_avatar, depvar='absolute_180_angles', subject='subject_id', within=['avatar_id']).fit())

ValueError: Data is unbalanced.